<a href="https://colab.research.google.com/github/piehyun/AI/blob/main/%EC%98%A4%EB%AA%A9__py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import random

def initialize_board(starting_stone):
    board = [['X'] * 19 for _ in range(19)]
    return board, starting_stone

def print_board(board):
    for row in board:
        print(" ".join(map(str, row)))

def player_move(board, time_limit):
    start_time = time.time()

    while time.time()-start_time <= time_limit:
        try:
            move = input("Enter your move (row, col): ").split(',')
            row, col = int(move[0]), int(move[1])
            if board[row][col] != 'X':
                print("Invalid move. That position is already taken.")
                continue
            else:
                return row, col
        except (ValueError, IndexError):
            print("Invalid input. Please enter row and column numbers.")
    print("Time's up! Place a stone randomly.")
    empty_positions = [(i, j) for i in range(19) for j in range(19) if board[i][j] == 'X']
    if empty_positions:
        row, col = random.choice(empty_positions)
        return row, col
    else:
        print("No empty positions left.")

def ai_move(board, time_limit):
    start_time = time.time()

    best_move = alpha_beta_search(board, 2, float('-inf'), float('inf'), True)[1]

    if time.time()-start_time <= time_limit:
        return best_move
    else:
        print("Time's up! Place a stone randomly.")
        empty_positions = [(i, j) for i in range(19) for j in range(19) if board[i][j] == 'X']
        if empty_positions:
            row, col = random.choice(empty_positions)
            return row, col
        else:
            print("No empty positions left.")

def evaluate(board):
    def count_stones_in_direction(x, y, dx, dy, stone):
        count = 0
        nx, ny = x + dx, y + dy
        while 0 <= nx < 19 and 0 <= ny < 19 and board[nx][ny] == stone:
            count += 1
            nx, ny = nx + dx, ny + dy
        return count

    def score_line(line):
        score = 0
        length = len(line)
        i = 0
        while i < length:
            if line[i] == 'W':
                count_w = 1
                j = i + 1
                while j < length and line[j] == 'W':
                    count_w += 1
                    j += 1
                if count_w == 5:
                    return 1000
                elif count_w == 4:
                    score += 800
                elif count_w == 3:
                    score += 600
                i = j
            elif line[i] == 'B':
                count_b = 1
                j = i + 1
                while j < length and line[j] == 'B':
                    count_b += 1
                    j += 1
                if count_b == 5:
                    return -1000
                elif count_b == 4:
                    score -= 800
                elif count_b == 3:
                    score -= 600
                i = j
            else:
                i += 1
        return score

    total_score = 0
    for i in range(19):
        total_score += score_line(board[i])
    for j in range(19):
        total_score += score_line([board[i][j] for i in range(19)])
    for i in range(15):
        for j in range(15):
            line = [board[i+k][j+k] for k in range(5)]
            total_score += score_line(line)
    for i in range(15):
        for j in range(4, 19):
            line = [board[i+k][j-k] for k in range(5)]
            total_score += score_line(line)

    return total_score

def alpha_beta_search(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or game_over(board):
        return evaluate(board), None

    if maximizing_player:
        value = float('-inf')
        best_move = None
        for move in possible_moves(board):
            board[move[0]][move[1]] = 'W'
            score = alpha_beta_search(board, depth - 1, alpha, beta, False)[0]
            board[move[0]][move[1]] = 'X'
            if score > value:
                value = score
                best_move = move
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value, best_move
    else:
        value = float('inf')
        best_move = None
        for move in possible_moves(board):
            board[move[0]][move[1]] = 'B'
            score = alpha_beta_search(board, depth - 1, alpha, beta, True)[0]
            board[move[0]][move[1]] = 'X'
            if score < value:
                value = score
                best_move = move
            beta = min(beta, value)
            if alpha >= beta:
                break
        return value, best_move

def possible_moves(board):
    moves = []
    for i in range(19):
        for j in range(19):
            if board[i][j] == 'X':
                moves.append((i, j))
    return moves

def game_over(board):
    def check_omok(x, y):
        directions = [(1, 0), (0, 1), (1, 1), (1, -1)]
        for dx, dy in directions:
            count = 1
            nx, ny = x + dx, y + dy
            while 0 <= nx < 19 and 0 <= ny < 19 and board[nx][ny] == board[x][y]:
                count += 1
                nx, ny = nx + dx, ny + dy
            nx, ny = x - dx, y - dy
            while 0 <= nx < 19 and 0 <= ny < 19 and board[nx][ny] == board[x][y]:
                count += 1
                nx, ny = nx - dx, ny - dy
            if count >= 5:
                return True
        return False

    if all(board[i][j] != 'X' for i in range(19) for j in range(19)):
        return True

    for i in range(19):
        for j in range(19):
            if board[i][j] != 'X' and check_omok(i, j):
                return True

    return False

def play_game():
    starting_stone = input("Choose your starting stone ('B' or 'W'): ").upper()
    if starting_stone not in ('B', 'W'):
        print("Invalid choice. Please choose 'B' or 'W'.")
        return
    board, starting_stone = initialize_board(starting_stone)
    print("Initial Board:")
    print_board(board)

    while True:
        if starting_stone == 'B':
            print("Player's Turn (Black):")
            player_row, player_col = player_move(board, 10)
            board[player_row][player_col] = 'B'
            print("Player's Move:")
            print_board(board)

            if game_over(board):
                print("Player wins!")
                break

            print("AI's Turn (White):")
            ai_row, ai_col = ai_move(board, 10)
            board[ai_row][ai_col] = 'W'
            print("AI's Move:")
            print_board(board)

            if game_over(board):
                print("AI wins!")
                break
        else:
            print("AI's Turn (Black):")
            ai_row, ai_col = ai_move(board, 10)
            board[ai_row][ai_col] = 'B'
            print("AI's Move:")
            print_board(board)

            if game_over(board):
                print("AI wins!")
                break

            print("Player's Turn (White):")
            player_row, player_col = player_move(board, 10)
            board[player_row][player_col] = 'W'
            print("Player's Move:")
            print_board(board)

            if game_over(board):
                print("Player wins!")
                break

play_game()


Choose your starting stone ('B' or 'W'): W
Initial Board:
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
AI's Turn (Black):
AI's Move:
B X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
X X X X X X X X X X X X X X X X X X X
